In [6]:
import open3d as o3d
import numpy as np
# Load the PLY file
ply_file = "C:/Users/nguye/Desktop/3d/test_data/test.ply"
pcd = o3d.io.read_point_cloud(ply_file)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
def create_infinite_axes():
    """
    Creates a set of long lines along the X, Y, and Z axes to simulate infinite axes.
    """
    # Define start and end points for each axis (very long lines)
    length = 100  # Large number to simulate infinite axes
    points = np.array([
        [-length, 0, 0], [length, 0, 0],  # X-axis (red)
        [0, -length, 0], [0, length, 0],  # Y-axis (green)
        [0, 0, -length], [0, 0, length]   # Z-axis (blue)
    ])
    
    # Define line connections (pairs of points)
    lines = [[0, 1], [2, 3], [4, 5]]
    
    # Define colors: Red for X, Green for Y, Blue for Z
    colors = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]

    # Create LineSet
    axis_lines = o3d.geometry.LineSet()
    axis_lines.points = o3d.utility.Vector3dVector(points)
    axis_lines.lines = o3d.utility.Vector2iVector(lines)
    axis_lines.colors = o3d.utility.Vector3dVector(colors)

    return axis_lines

In [8]:
geometry = [create_infinite_axes()]
geometry.append(pcd)
o3d.visualization.draw_geometries(geometry)

In [9]:
def look_from_positive_z(vis):
    ctr = vis.get_view_control()
    ctr.set_front([0, 0, 1])
    ctr.set_lookat([0, 0, 0])
    ctr.set_up([0, 1, 0])
    ctr.set_zoom(0.1)
    return False

def look_from_positive_x(vis):
    ctr = vis.get_view_control()
    ctr.set_front([1, 0, 0])
    ctr.set_lookat([0, 0, 0])
    ctr.set_up([0, 0, 1])
    ctr.set_zoom(0.1)
    return False

def look_from_positive_y(vis):
    ctr = vis.get_view_control()
    ctr.set_front([0, 1, 0])
    ctr.set_lookat([0, 0, 0])
    ctr.set_up([0, 0, 1])
    ctr.set_zoom(0.1)
    return False

key_to_callback = {}
key_to_callback[ord("Z")] = look_from_positive_z
key_to_callback[ord("X")] = look_from_positive_x
key_to_callback[ord("Y")] = look_from_positive_y


o3d.visualization.draw_geometries_with_key_callbacks(geometry, key_to_callback)

In [10]:
test_data = "frame000001.png,16.07672148106882,-13.36208053348467,21.43087990863757,-43.76176907447967,57.05063527217181,-1.699805914025459,29.80963545052193,-8.607111067081718e-004,6.034445625206664e-003,0.1463257180718982,-0.7163134720697377,1.231617563039704,0,0,0"

def parse_line(line):
    line = line.strip().split(",")
    filename = line[0]
    position = np.array([float(x) for x in line[1:4]])
    rotation = np.array([float(x) for x in line[4:7]])
    return filename, position, rotation

filename, position, rotation = parse_line(test_data)
print(f"Filename: {filename}")
print(f"Position: {position}")
print(f"Rotation: {rotation}")
# Create a rectangle at the given position and apply the rotation
rectangle = o3d.geometry.TriangleMesh.create_box(width=5.0, height=10.0, depth=5.0)
rectangle.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle(rotation[:3]))
rectangle.translate(position)

# Add the rectangle to the geometry list
geometry.append(rectangle)

# Visualize the updated geometry
o3d.visualization.draw_geometries_with_key_callbacks(geometry, key_to_callback)


Filename: frame000001.png
Position: [ 16.07672148 -13.36208053  21.43087991]
Rotation: [-43.76176907  57.05063527  -1.69980591]


In [46]:
import numpy as np
import open3d as o3d
from scipy.spatial.transform import Rotation as R

def create_camera_visualization(position, rotation_angles, size=1.0):
    """
    Create a 3D visualization of the camera position & orientation.
    
    Parameters:
    - position: (x, y, z) Camera world coordinates
    - rotation_angles: (omega, phi, kappa) in degrees
    - size: Scaling factor for the camera visualization
    
    Returns:
    - Open3D camera coordinate frame object
    """
    # Convert rotation angles to rotation matrix
    # rotation_matrix = R.from_euler('zxy', rotation_angles, degrees=True).as_matrix()
    rotation_matrix = euler_to_rotation_matrix(*rotation_angles)

    # Create a 4x4 transformation matrix
    camera_transform = np.eye(4)
    camera_transform[:3, :3] = rotation_matrix
    camera_transform[:3, 3] = position

    # Create Open3D coordinate frame for the camera
    camera_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)
    
    # Apply transformation to place the camera
    camera_frame.transform(camera_transform)
    
    return camera_frame


In [47]:
def create_camera_frustum(fov_degrees=60, aspect_ratio=1.5, scale=1.0):
    """
    Creates a 3D frustum representation of the camera's field of view.

    Parameters:
    - fov_degrees: Field of View (FoV) in degrees
    - aspect_ratio: Aspect ratio (width/height)
    - scale: Frustum size scaling factor

    Returns:
    - Open3D frustum visualization as a LineSet object
    """
    import math

    # Convert FoV to radians
    fov_radians = math.radians(fov_degrees / 2.0)

    # Define frustum corners (near plane)
    near = scale
    far = scale * 2
    h = math.tan(fov_radians) * near
    w = h * aspect_ratio

    # Frustum vertices
    vertices = [
        [0, 0, 0],  # Camera center (apex)
        [-w, -h, -near], [w, -h, -near], [w, h, -near], [-w, h, -near],  # Near plane
        [-w * 2, -h * 2, -far], [w * 2, -h * 2, -far], [w * 2, h * 2, -far], [-w * 2, h * 2, -far]  # Far plane
    ]

    # Frustum edges
    edges = [
        (0, 1), (0, 2), (0, 3), (0, 4),  # Apex to near plane
        (1, 2), (2, 3), (3, 4), (4, 1),  # Near plane edges
        (1, 5), (2, 6), (3, 7), (4, 8),  # Near to far connections
        (5, 6), (6, 7), (7, 8), (8, 5)   # Far plane edges
    ]

    # Create Open3D LineSet
    frustum = o3d.geometry.LineSet()
    frustum.points = o3d.utility.Vector3dVector(vertices)
    frustum.lines = o3d.utility.Vector2iVector(edges)

    return frustum


In [48]:
# Example Camera Data
camera_position = (-9.886853564542303,-10.59119235780782,27.4286920728011)  # (X, Y, Z)
camera_rotation = (41.69092292632279,58.46206628747401,3.678327983550952)  # (Omega, Phi, Kappa)


# Load point cloud (replace with your path)
# pcd = o3d.io.read_point_cloud("your_pointcloud.ply")

# Create a camera visualization
camera_vis = create_camera_visualization(camera_position, camera_rotation, size=2.0)

# Render
o3d.visualization.draw_geometries([pcd, camera_vis, create_infinite_axes()])


41.69092292632279 58.46206628747401 3.678327983550952
0.7276438732595221 1.0203555442355992 0.06419893428120794


In [14]:
def euler_to_rotation_matrix(yaw, pitch, roll):
    """
    Compute rotation matrix based on RealityCapture's Yaw (Z), Pitch (Y), Roll (X).
    """
    # Convert degrees to radians
    print(yaw, pitch, roll)
    yaw, pitch, roll = np.radians([yaw, pitch, roll])
    print(yaw, pitch, roll)
    # Compute sines and cosines
    cx, cy, cz = np.cos([roll, pitch, yaw])
    sx, sy, sz = np.sin([roll, pitch, yaw])

    # RealityCapture-style rotation matrix
    R = np.array([
        [ cx * cz + sx * sy * sz, -cx * sz + cz * sx * sy, -cy * sx ],
        [-cy * sz,                -cy * cz,                -sy     ],
        [ cx * sy * sz - cz * sx,  cx * cz * sy + sx * sz, -cx * cy ]
    ])
    
    return R

In [ ]:
# Create camera transformation matrix
def get_camera_transform(position, rotation_angles):
    """
    Returns a 4x4 transformation matrix for the camera.
    
    Parameters:
    - position: (x, y, z)
    - rotation_angles: (omega, phi, kappa) in degrees
    
    Returns:
    - 4x4 transformation matrix as a NumPy array
    """
    rotation_matrix = euler_to_rotation_matrix(*rotation_angles)
    # rotation_matrix = R.from_euler('xyz', rotation_angles, degrees=True).as_matrix()


    R_fix = np.array([
        [1,  0,  0],   # No change to X
        [0,  1,  0],   # Swap Z with Y
        [0,  0,  -1],   # Swap Y with -Z
    ])

    # R_fix = o3d.geometry.get_rotation_matrix_from_xyz([-np.pi / 2, 0, 0])  # Rotate to align with Z


    # fix_rotation = R.from_euler('x', 90, degrees=True).as_matrix()  # Rotate around X
    rotation_matrix = rotation_matrix @ R_fix  # Apply correction
    
    transform = np.eye(4)
    transform[:3, :3] = rotation_matrix
    transform[:3, 3] = position
    
    return transform


In [ ]:
# Create a camera frustum
frustum = create_camera_frustum()
camera_transform = get_camera_transform(camera_position, camera_rotation)

frustum.transform(camera_transform)


# **Add Look-At Direction Arrow**
look_at_length = 3.0  # Arrow length
look_at_direction = camera_transform[:3, 2]  # The Z-axis is the view direction
look_at_end = camera_position + look_at_length * look_at_direction

# Create an arrow mesh for look-at visualization
arrow = o3d.geometry.TriangleMesh.create_arrow(cylinder_radius=0.1, cone_radius=0.2,
                                               cylinder_height=12.5, cone_height=0.5)
arrow.paint_uniform_color([1, 0, 0])  # Red arrow

# Position & orient the arrow
arrow.translate(camera_position)
arrow_direction = camera_position - look_at_end
arrow_rotation = R.align_vectors([arrow_direction], [[0, 0, 1]])[0].as_matrix()  # Align to Z-axis
arrow.rotate(arrow_rotation, center=camera_position)

# Render scene with point cloud and frustum
o3d.visualization.draw_geometries([pcd, frustum, arrow])

In [ ]:
def read_camera_poses_from_file(filename):
    """
    Read camera poses from a text file.
    
    Each line should contain the following information:
    - Filename: Image filename
    - Position: (x, y, z) in world coordinates
    - Rotation: (omega, phi, kappa) in degrees
    
    Returns:
    - List of tuples (filename, position, rotation)
    """
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    camera_poses = []
    for line in lines:
        if("#name" in line):
            continue
        filename, position, rotation = parse_line(line)
        camera_poses.append((filename, position, rotation))
    
    return camera_poses

In [ ]:
camera_poses = read_camera_poses_from_file("C:/Users/nguye/Desktop/3d/test_data/reg.csv")

In [ ]:
print(camera_poses[0])

('frame000201.png', array([  3.04450497, -28.32376767,  29.71072178]), array([-0.82244069, 64.5725128 ,  6.50867737]))


In [ ]:
def render_frustum_base_on_camera_poses(camera_poses, point_cloud, scale=1.0):
    """
    Render camera frustums based on a list of camera poses.
    
    Parameters:
    - camera_poses: List of tuples (filename, position, rotation)
    - point_cloud: Open3D point cloud object
    - scale: Frustum scaling factor
    
    Returns:
    - List of Open3D geometries to render
    """
    geometries = [point_cloud]
    
    for _, position, rotation in camera_poses:
        # Create camera frustum
        frustum = create_camera_frustum(scale=scale)
        camera_transform = get_camera_transform(position, rotation)
        frustum.transform(camera_transform)

        scale_down = 0.1
        frustum.scale(scale_down, center=position)
        
        # Add to the list of geometries
        geometries.append(frustum)
    
    return geometries

In [21]:
def generate_axes(scale):
    points = [
        [0, 0, 0],
        [scale, 0, 0],
        [0, scale, 0],
        [0, 0, scale],
    ]
    lines = [
        [0, 1],
        [0, 2],
        [0, 3]
    ]
    colors = [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set


In [32]:
# geometry = render_frustum_base_on_camera_poses(camera_poses, pcd, scale=1.0)
# geometry.append(create_infinite_axes())
o3d.visualization.draw_geometries([pcd, generate_axes(50)])